In [50]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 1000, 'display.max_colwidth', 1000, 'display.max_rows',1000)

# Data Read "elon-musk-tweets" file

In [51]:
df1 = pd.read_csv('data/elon-musk-tweets.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13610 entries, 0 to 13609
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            13590 non-null  object
 1   username        13610 non-null  object
 2   linktotweet     13610 non-null  object
 3   tweetembedcode  13610 non-null  object
 4   createdat       13610 non-null  object
dtypes: object(5)
memory usage: 531.8+ KB


In [52]:
start = '2020-03-15'
end = '2022-03-14'

In [53]:
df1 = df1[(df1.createdat >= start) & (df1.createdat <= end)][['text','createdat']]\
    .sort_values(by='createdat', ascending=False)\
    .reset_index(drop = True)
df1.rename(columns={'text':'Content', 'createdat':'Time'}, inplace = True)

def date_time(x):
    x = str.split(x,sep='T')
    time = str.split(x[0],sep="-")
    year = int(time[0])
    mon = int(time[1])
    day = int(time[2])

    time = str.split(x[1],sep=":")
    hr = int(time[0])
    min = int(time[1])
    sec = int(time[2])
    sec = int('0')

    date = datetime(year=year,month=mon,day=day,hour=hr, minute=min, second=sec)
    
    return date

df1['Time'] = df1.Time.apply(lambda x: date_time(x))

df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6872 entries, 0 to 6871
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Content  6862 non-null   object        
 1   Time     6872 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 107.5+ KB


# Data Read "Elon Musk 2" file

In [57]:
df2 = pd.read_csv('data/Elon Musk 2.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7918 entries, 0 to 7917
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PostID         7918 non-null   int64 
 1   Time           7918 non-null   object
 2   TweetUrl       7918 non-null   object
 3   Content        7918 non-null   object
 4   UserID         7918 non-null   int64 
 5   UserName       7918 non-null   object
 6   RetweetNum     7918 non-null   int64 
 7   LikeNum        7918 non-null   int64 
 8   ReplyNum       7918 non-null   int64 
 9   UserHandle     7918 non-null   object
 10  UserUrl        7918 non-null   object
 11  Location       112 non-null    object
 12  UserID_PostID  7918 non-null   object
 13  CurrentPage    7918 non-null   int64 
dtypes: int64(6), object(8)
memory usage: 866.2+ KB


In [58]:
from datetime import datetime
months_in_year = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
months_in_year.index('Jan')

df2 = df2[['Content','Time']].sort_values(by='Time')

def time_convert(x):
    x = str.split(x)
    year = int(x[-1])
    mon = int(months_in_year.index(x[1])+1)
    day = int(x[2])
    time = x[3].split(':')
    hr = int(time[0])
    min = int(time[1])
    sec = int(time[2])
    sec = int('0')
    date = datetime(year=year,month=mon,day=day,hour=hr, minute=min, second=sec)
    return date

df2['Time'] = df2.Time.apply(lambda x: time_convert(x))

df2 = df2[(df2.Time >= start) & (df2.Time <= end)].sort_values(by='Time',ascending=False)
df2.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7136 entries, 511 to 7129
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Content  7136 non-null   object        
 1   Time     7136 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 167.2+ KB


# Combine two dataset

In [81]:
data = df1.append(df2).drop_duplicates(subset='Time').reset_index(drop=False)
print(("min Time: "+str(data.Time.min())))
print(("max Time: "+str(data.Time.max())))
data.to_csv("data/combined_data.csv")
data.info()

min Time: 2020-03-15 03:02:00
max Time: 2022-03-11 19:41:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10786 entries, 0 to 10785
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   index    10786 non-null  int64         
 1   Content  10776 non-null  object        
 2   Time     10786 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 252.9+ KB


/var/folders/0f/s8khq6c91r5dp2kn1ggjqb800000gn/T/ipykernel_42319/3906135938.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = df1.append(df2).drop_duplicates(subset='Time').reset_index(drop=False)


## Tweet Content Cleaning

In [ ]:
def sentence_to_wordlist(sentence, remove_stopwords=False):
    # 1. Remove non-letters
    sentence_text = re.sub(r'[^\w\s]','', sentence)
    # 2. Remove all numbers
    sentence_text = re.sub(r'[0-9]+', '', sentence_text)
    # 3. Convert words to lower case and split them
    words = sentence_text.lower().split()
    # 4. remove website address   
    sentence = re.sub(r'^https?:\/\/.*[\r\n]*', '', sentence, flags=re.DOTALL)
    #sentence_text = re.sub(r'^https?:\/\/.*[\r\n]*', '', flags=re.MULTILINE)
    #6. drop the https
    #sentence = re.sub(r'[%https]', '', sentence)
    # 4. drop http
    p1=re.compile(r'http\S+', re.DOTALL)
    sentence = re.sub(p1, '', sentence)
    # 5. drop https
    p1=re.compile(r'https\S+', re.DOTALL)
    sentence = re.sub(p1, '', sentence)
    # 5. Stemming
    words = [stemmer.stem(w) for w in words] 
    # 6. Lemmatizing
    words = [lemmatizer.lemmatize(word) for word in words]
    # 7. Return a list of words
    return(words)